In [1]:
from decouple import config

In [2]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [3]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [4]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key

params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY",
}
url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=5min&apikey={api_key}'.format(
    **params
)
r = requests.get(url)
data = r.json()

In [6]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (5min)'

In [18]:
results = data[dataset_key]
# for timestamp_str in results.keys():
#     print(timestamp_str, results.get(timestamp_str))

In [19]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2025-01-30 19:55:00'

In [21]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '244.5600',
 '2. high': '244.9000',
 '3. low': '244.4501',
 '4. close': '244.7900',
 '5. volume': '52216'}

In [30]:
import pytz
from datetime import datetime
from decimal import Decimal

def transfrom_alpha_vantage_result(timestamp_str, result):
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }

In [31]:
transfrom_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('244.5600'),
 'close_price': Decimal('244.7900'),
 'high_price': Decimal('244.9000'),
 'low_price': Decimal('244.4501'),
 'number_of_trades': None,
 'volume': 52216,
 'volume_weighted_average': None,
 'time': datetime.datetime(2025, 1, 31, 0, 55, tzinfo=<UTC>)}

In [32]:
for timestamp_str in results.keys():
    new_data = transfrom_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('244.5600'), 'close_price': Decimal('244.7900'), 'high_price': Decimal('244.9000'), 'low_price': Decimal('244.4501'), 'number_of_trades': None, 'volume': 52216, 'volume_weighted_average': None, 'time': datetime.datetime(2025, 1, 31, 0, 55, tzinfo=<UTC>)}
{'open_price': Decimal('244.4300'), 'close_price': Decimal('244.5600'), 'high_price': Decimal('244.6567'), 'low_price': Decimal('244.2000'), 'number_of_trades': None, 'volume': 30082, 'volume_weighted_average': None, 'time': datetime.datetime(2025, 1, 31, 0, 50, tzinfo=<UTC>)}
{'open_price': Decimal('244.2600'), 'close_price': Decimal('244.4800'), 'high_price': Decimal('244.4800'), 'low_price': Decimal('244.2000'), 'number_of_trades': None, 'volume': 15441, 'volume_weighted_average': None, 'time': datetime.datetime(2025, 1, 31, 0, 45, tzinfo=<UTC>)}
{'open_price': Decimal('244.5000'), 'close_price': Decimal('244.2900'), 'high_price': Decimal('244.7500'), 'low_price': Decimal('244.2000'), 'number_of_trades': None,